In [ ]:
import ultralytics
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import torch    
import cv2
import numpy as np
import matplotlib.pyplot as plt
import requests
import torchvision.transforms as transforms
from PIL import Image
import io
import os
import shutil

plt.rcParams["figure.figsize"] = [3.0, 3.0]

from yolo_cam.eigen_cam import EigenCAM
from yolo_cam.utils.image import show_cam_on_image, scale_cam_image

## one image
#### -You may get an innaccurate output at first. Re-running the cell will do the trick

In [ ]:
model = YOLO('../../model/weights/yolov8_weights.pt')

In [ ]:
img = cv2.imread('images/test.jpg')
img = cv2.resize(img, (640, 640))
rgb_img = img.copy()
img = np.float32(img) / 255

In [ ]:
# 레이어 설정
target_layers =[model.model.model[-4]]

In [ ]:
cam = EigenCAM(model, target_layers,task='od')
grayscale_cam = cam(rgb_img)[0, :, :]
cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
plt.imshow(cam_image)
plt.show()

In [ ]:
g_scale = np.stack([grayscale_cam] * 3, axis=2)
plt.imshow(g_scale)

In [ ]:
im = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
Image.fromarray(np.hstack((im, cam_image)))

## multiple images

In [ ]:
# model load
model = YOLO('../../model/weights/yolov8_weights.pt')

# 이미지 파일이 들어있는 폴더 경로 설정
input_folder = './images'

# 결과 이미지를 저장할 폴더 경로 설정
output_folder = './result'

if os.path.exists(output_folder):
    shutil.rmtree(output_folder)
    # make dir if not exist
    os.makedirs(output_folder, exist_ok=True)

# input_folder에 있는 모든 이미지 파일들을 가져오기
image_files = os.listdir(input_folder)

# 레이어 설정
target_layers = [model.model.model[-4]]

for image_file in image_files:
    # 이미지 파일 경로 생성
    image_path = os.path.join(input_folder, image_file)
    
    # 이미지 불러오기
    img = cv2.imread(image_path)
    img = cv2.resize(img, (640, 640))
    rgb_img = img.copy()
    img = np.float32(img) / 255

    cam = EigenCAM(model, target_layers, task='od')
    grayscale_cam = cam(rgb_img)[0, :, :]
    grayscale_cam = cam(rgb_img)[0, :, :]
    cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)

    # 결과 이미지 파일 경로 설정
    print(os.path.join(output_folder, image_file))
    output_path = os.path.join(output_folder, image_file)
    
    # 결과 이미지 저장
    im = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
    result_image = Image.fromarray(np.hstack((im, cam_image)))
    result_image.save(output_path)